# Ex 04 A/B test

In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("../data/checking-logs.sqlite")
cur = con.cursor()


In [4]:
pd.read_sql("""
            SELECT
                (CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600 as diff,
                CASE
                   WHEN first_commit_ts <= first_view_ts
                        THEN 'before'
                   ELSE 'after'
                END as time
            FROM test t JOIN deadlines d ON d.labs = t.labname  
            WHERE labname != 'project1'
            """
            ,con)

,diff,time
0,-202,after
1,-201,after
2,-200,after
3,-193,after
4,-148,before
5,-148,before
6,-126,before
7,-99,before
8,-98,before
9,-81,after


In [3]:
test_results = pd.read_sql("""
            SELECT
                AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as avg_diff,
                CASE
                   WHEN first_commit_ts <= first_view_ts
                        THEN 'before'
                   ELSE 'after'
                END as time
            FROM test t JOIN deadlines d ON d.labs = t.labname  
            WHERE labname != 'project1'
            GROUP BY time
            """
            ,con)
test_results

,avg_diff,time
0,-103.40625,after
1,-60.56250,before


In [4]:
control_results = pd.read_sql("""
            SELECT
                AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as avg_diff,
                CASE
                   WHEN first_commit_ts <= first_view_ts
                        THEN 'before'
                   ELSE 'after'
                END as time
            FROM control t JOIN deadlines d ON d.labs = t.labname  
            WHERE labname != 'project1'
            GROUP BY time
            """
            ,con)
control_results

,avg_diff,time
0,-108.928571,after
1,-99.464286,before


Время между первым коммитом и дедлайном значимо увеличилось по сравнению с увеличением в контрольной группе (у нас нет оснований отклонить нулевую гипотезу в контрольной группе). Страница положительно влияет на поведение студентов, они раньше стали приступать к заданиям.

In [5]:
con.close()